In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
train

In [ ]:
X_test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
X_test

In [ ]:
X_data_train = train.drop(['target', 'id'], axis=1)
y_data_train = train.target

In [ ]:
X_test.drop(['id'], axis=1, inplace=True)

In [ ]:
df_train = X_data_train.to_numpy(dtype=np.float32)

In [ ]:
y_train = y_data_train.to_numpy()
y_train = torch.from_numpy(y_train)

In [ ]:
df_train = torch.from_numpy(df_train)

In [ ]:
df_test = X_test.to_numpy(dtype=np.float32)
df_test = torch.from_numpy(df_test)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(Net, self).__init__()
        
        self.fc1 = torch.nn.Linear(100, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, 2)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
net = Net(35)

In [ ]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(net.parameters(), 
                             lr=1.0e-3)

In [ ]:
batch_size = 60000

for epoch in range(1000):
    order = np.random.permutation(len(df_train))
    for start_index in range(0, len(df_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = df_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = net.forward(x_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        print(epoch/100)

In [ ]:
test_preds = net.forward(df_test)
test_preds = test_preds.argmax(dim=1)
test_preds

In [ ]:
y_test = test_preds.numpy()

In [ ]:
ind = np.linspace(600000, 1139999,num=540000, dtype=int)
y_test = pd.DataFrame({'id': ind, 'target': y_test})
print(y_test.target.value_counts())
# y_test = y_test.set_index('id')
print(y_test)
y_test.to_csv('y_test.csv', index=None)

In [ ]:
y_test.to_csv('y_test.csv', index=None)